In [1]:
import os, sys
rmg_path = os.getenv('RMGpy')
if rmg_path and rmg_path not in sys.path:
    sys.path.insert(1,rmg_path)

import os
import logging # doesn't work well in a notebook, without some fu
import rmgpy
import rmgpy.rmg
import rmgpy.rmg.input
#from rmgpy.display import display
from IPython.display import display
from rmgpy.chemkin import loadChemkinFile, readSpeciesBlock, readThermoBlock, readReactionsBlock, removeCommentFromLine
from rmgpy.data.thermo import Entry, saveEntry
from rmgpy.data.base import Entry as kinEntry
from rmgpy.data.kinetics.common import saveEntry as kinSaveEntry
from rmgpy.molecule import Molecule
from rmgpy.rmg.model import Species
from rmgpy.reaction import Reaction
from rmgpy.kinetics import Arrhenius
from rmgpy.cantherm.output import prettify
from rmgpy.rmg.main import RMG, initializeLog
from rmgpy.molecule.draw import MoleculeDrawer
import cPickle as pickle
import pandas as pd



/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
with open("kinetics.pkl", "r") as f:
    importerKinetics = pickle.load(f)

In [3]:
importerKinetics

{Reaction(reactants=[Species(label="NC3H7", molecule=[Molecule(SMILES="[CH2]CC")]), Species(label="C9H18-1", molecule=[Molecule(SMILES="C=CCCCCCCC")])], products=[Species(label="C12H25-5", molecule=[Molecule(SMILES="CCCC[CH]CCCCCCC")])]): {'CombFlame2013/17-Malewicki': Arrhenius(A=(1e+11,'cm^3/(mol*s)'), n=0, Ea=(8200,'cal/mol'), T0=(1,'K')),
  'PCI2013/361-Malewicki': Arrhenius(A=(1e+11,'cm^3/(mol*s)'), n=0, Ea=(8200,'cal/mol'), T0=(1,'K'))},
 Reaction(reactants=[Species(label="R93C4H9O4UP", molecule=[Molecule(SMILES="CC(CCOO)O[O]")])], products=[Species(label="R2OH", molecule=[Molecule(SMILES="[OH]")]), Species(label="C4H8O3KP", molecule=[Molecule(SMILES="O=CCCCOO")])], reversible=False, duplicate=True): {'PCI2013/297-Herbinet': MultiArrhenius(arrhenius=[Arrhenius(A=(5e+09,'s^-1'), n=1, Ea=(35500,'cal/mol'), T0=(1,'K')), Arrhenius(A=(3.3e+09,'s^-1'), n=1, Ea=(32500,'cal/mol'), T0=(1,'K')), Arrhenius(A=(5.7e+08,'s^-1'), n=1, Ea=(23500,'cal/mol'), T0=(1,'K'))])},
 Reaction(reactants=[S

In [4]:
for key in importerKinetics:
    try:
        print importerKinetics[key]['AutoTST-OOHabstraction']
        del importerKinetics[key]['AutoTST-OOHabstraction']
    except:
        pass
    


Arrhenius(A=(1.50381e-08,'cm^3/(mol*s)'), n=5.67296, Ea=(15.8252,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 2.65616, dn = +|- 0.128207, dEa = +|- 0.705266 kJ/mol""")
Arrhenius(A=(0.0114732,'cm^3/(mol*s)'), n=3.89426, Ea=(57.6883,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 1.07028, dn = +|- 0.00891434, dEa = +|- 0.0490378 kJ/mol""")
Arrhenius(A=(0.000274848,'cm^3/(mol*s)'), n=4.67067, Ea=(21.5744,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 1.59853, dn = +|- 0.0615635, dEa = +|- 0.338661 kJ/mol""")
Arrhenius(A=(5.68621e-05,'cm^3/(mol*s)'), n=4.6047, Ea=(3.1184,'kJ/mol'), T0=(1,'K'), Tmin=(303.03,'K'), Tmax=(2500,'K'), comment="""Fitted to 59 data points; dA = *|/ 1.45361, dn = +|- 0.049091, dEa = +|- 0.27005 kJ/mol""")
Arrhenius(A=(0.000337358,'cm^3/(mol*s)'), n=4.70263, Ea=(46.2109,'kJ/mol'), T0=(1,'K'), T

In [5]:
pd.DataFrame(importerKinetics)

,c4h7oh1-4 + o2 <=> c4h6oh1-32 + ho2,IC4H9T + C4H9CHO => IC4H10 + C3H6 + CH2CHO,C5H7 + C4H9CHO => C5H8 + C3H6 + CH2CHO,HCO3 + C4H9CHO => HCO3H + C3H6 + CH2CHO,IC8H18 + NC7H15 <=> NEOC7H16 + DC8H17,X135C6H8 + CH3 <=> CEC7H11A,CH3CO3 + C4H9CHO => CH3CO3H + C3H6 + CH2CHO,nc8h17cho + o2 <=> nc8h17co + ho2,NC3H7OO + C4H9CHO => C3H7OOH + C3H6 + CH2CHO,IC3H7OO + C4H9CHO => C3H7OOH + C3H6 + CH2CHO,...,C6H6 + O <=> C6H5O + H,R114C10H21O2P <=> R121C10H21O2P,C7H12-15 <=> aC3H5 + SAXC4H7,c4h6 + c4h7-3 <=> ic4h5 + c4h8-1,C6H6 + OH <=> C6H5 + H2O,ch3coch2o <=> ch3co + ch2o,IC4P-QOOH => IC4H9P-OO,IC4H9T-OO => IC4T-QOOH,NC11H24 + O2 <=> C11H23-6 + HO2,sC3H5 + O2 <=> CH3CO + CH2O
AramcoMech_1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AramcoMech_2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Arrhenius(A=(2.2e+13,'cm^3/(mol*s)'), n=0, Ea=...",NaN,NaN,NaN,"Arrhenius(A=(1.2,'cm^3/(mol*s)'), n=4.1, Ea=(-...","Arrhenius(A=(5.872e+20,'s^-1'), n=-2.4218, Ea=...",NaN,NaN,NaN,NaN
Biomass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chernov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Arrhenius(A=(2.2e+13,'cm^3/(mol*s)'), n=0, Ea=...",NaN,NaN,NaN,"Arrhenius(A=(1.45e+13,'cm^3/(mol*s)'), n=0, Ea...",NaN,NaN,NaN,NaN,NaN
CombFlame2012/2028-Sarathy,"Arrhenius(A=(6e+12,'cm^3/(mol*s)'), n=0, Ea=(3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1315-Chang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/1541-Zhang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Arrhenius(A=(3.732e+20,'s^-1'), n=-2.176, Ea=(...",NaN,NaN,NaN,NaN
CombFlame2013/1609-Veloo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CombFlame2013/17-Malewicki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"Arrhenius(A=(1.2,'cm^3/(mol*s)'), n=4.1, Ea=(-...",NaN,NaN,NaN,"Arrhenius(A=(2e+13,'cm^3/(mol*s)'), n=0, Ea=(5...",NaN
CombFlame2013/1939-Cai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Arrhenius(A=(2.2e+13,'cm^3/(mol*s)'), n=0, Ea=...",NaN,NaN,NaN,"Arrhenius(A=(23400,'cm^3/(mol*s)'), n=2.68, Ea...",NaN,NaN,NaN,NaN,NaN


In [6]:
i = 0
comparisonKinetics = {}
for rxn in importerKinetics:
    
    
    mols = []
    reacts = []
    prods = []
    for reactant in rxn.reactants:
        reacts.append(reactant.molecule[0].toSMILES())
    for product in rxn.products:
        prods.append(product.molecule[0].toSMILES())
        
    if (("OO" in reacts and "[O]O" in prods) or ("OO" in prods and "[O]O" in reacts)) and len(reacts) + len(prods) == 4:
        print "Reaction WAS OOH abstraction"
        print rxn
        print
        comparisonKinetics[rxn] = importerKinetics[rxn]
        i += 1
    elif len(reacts) + len(prods) == 4:
        print "Reaction was not bimolecular"
        print rxn
        print
        
    else:
        print "Reaction was not OOH Abstraction"
        print rxn
        print 

Reaction was not OOH Abstraction
NC3H7 + C9H18-1 <=> C12H25-5

Reaction was not OOH Abstraction
R93C4H9O4UP => R2OH + C4H8O3KP

Reaction WAS OOH abstraction
NC12H25CHO + HO2 <=> NC12H25CO + H2O2

Reaction was not OOH Abstraction
R76C10H21O2P + O2 <=> R159C10H21O4UP

Reaction was not OOH Abstraction
C12OOH5-3 => C12O3-5 + OH

Reaction was not OOH Abstraction
C9OOH2-3 <=> C9H18-2 + HO2

Reaction was not OOH Abstraction
C2H5 + EDHF => C2H6 + C2H3COCH3 + C2H3

Reaction was not OOH Abstraction
C12OOH4-6 => C12O4-6 + OH

Reaction was not OOH Abstraction
C12OOH6-4 => C12O4-6 + OH

Reaction was not bimolecular
EC6H13 + CH3OH <=> IC6H14 + CH3O

Reaction was not bimolecular
NEOC5H12 + CH2CCH2OH <=> NEOC5H11 + C3H5OH

Reaction was not bimolecular
C3H5O + H <=> C2H3CHO + H2

Reaction was not bimolecular
mvk + oh <=> mvjk + h2o

Reaction was not OOH Abstraction
R270C6H13O2P => R122C3H7O2P + C3H6Y

Reaction was not OOH Abstraction
mb3oo + mb4oo => mb3o + mb4o + o2

Reaction was not OOH Abstraction
C

In [7]:
i

1047

In [8]:
with open("comparisonOOHRxns.py", "r") as f:
    lines = f.readlines()
rmgRxn = eval(lines[0].split('\n')[0])

split = rmgRxn.toChemkin(kinetics=False).split()
reactants = split[0], split[2]

a, b = rmgRxn.reactants
a


In [9]:
print "There were {} total imported reactions and {} bimolecular OOH reactions.".format(len(importerKinetics), len(comparisonKinetics))

There were 30204 total imported reactions and 1047 bimolecular OOH reactions.


In [10]:
weight_dict = {}
for item in comparisonKinetics.keys():
    
    try:
        weight = item.reactants[0].molecule[0].getMolecularWeight() + item.reactants[1].molecule[0].getMolecularWeight()
    except Exception as err:
        print str(item) + ': ' + str(err)
        
    weight_dict[item] = weight

pre_sorted = pd.DataFrame(weight_dict, index=[0]).transpose()
sorted_dict = pre_sorted.sort_values(0).transpose()
with open("comparisonOOHRxns.py", "w") as q:
    for rxn in sorted_dict.columns:
        q.write(rxn.__repr__() + '\n')



# This next portion is designed to find reactions that have already been run. tbh, this portion is kinda pointless cuz autoTST already looks for previously run reactions

```
nateDirectory = "/gss_gpfs_scratch/harms.n/QMscratch"
westGroupDirectory = "/gss_gpfs_scratch/westgroup/QMscratch"
dataDumpDirectory = "/gss_gpfs_scratch/westgroup/ScienceDataDump/Reactions/H_Abstraction"

nateDirectory = "/Users/nathan/Code/scratch_test/nate/"
westGroupDirectory = "/Users/nathan/Code/scratch_test/west/"
dataDumpDirectory = "/Users/nathan/Code/scratch_test/dump/"

nate_reactions = os.listdir(nateDirectory)
west_reactions = os.listdir(westGroupDirectory)
dump_reactions = os.listdir(dataDumpDirectory)

previous_reactions = []
paths = []
for reaction in nate_reactions:
    if reaction not in previous_reactions:
        previous_reactions.append(reaction)
        paths.append(os.path.join(nateDirectory, reaction, "output.py"))
    else:
        print "Duplicate reaction {}".format(reaction)
        
for reaction in west_reactions:
    if reaction not in previous_reactions:
        previous_reactions.append(reaction)
        paths.append(os.path.join(westGroupDirectory, reaction, "output.py"))
    else:
        print "Duplicate reaction {}".format(reaction)
        
for reaction in dump_reactions:
    if reaction not in previous_reactions:
        previous_reactions.append(reaction)
        try:
            paths.append(os.path.join(dataDumpDirectory, reaction, "m062x", "output.py"))
        except:
            print "Directory error"
    else:
        print "Duplicate reaction {}".format(reaction)
        
```
        

```
previous_reactions, paths

to_try = []
fail = 0
for path in paths:
    print path
    try:
        assert os.path.exists(path), "Doesn't have output.py file"
        to_try.append(path)
        
    except Exception as err:
        print "# ⚠️", path, "Gave error", str(err)
len(to_try)
    
for path in to_try:
    print path
    
    try:
        with open(path) as f:
            print "opened"
            lines = f.readlines()
        start = lines.index('kinetics(\n')
        print lines[start:]
        
        with 
    except Exception as err:
        print "# ⚠️", path, "Gave error", str(err)
        fail += 1
```    



```
## Using the reaciton filtering tool used in filter kinetics

to_try = []
fail = 0
success = 0
for path in paths:
    print path
    try:
        assert os.path.exists(path), "Doesn't have output.py file"
        to_try.append(path)
        
    except Exception as err:
        print "# ⚠️", reaction, "Gave error", str(err)
len(to_try)

for path in to_try:
    try:
        print path
        
        with open(path) as f:
            lines = f.readlines()
        start = lines.index('kinetics(\n')
        print
        print "# {} from {}".format(path)
        print ''.join(lines[start:])
        success += 1
        
    except Exception as err:
        print "# ⚠️", path, "Gave error", str(err)
        fail += 1
        
to_try
```

```
%%capture cap --no-stderr
for path in to_try:
    try:
        print path
        
        with open(path) as f:
            lines = f.readlines()
        start = lines.index('kinetics(\n')
        print
        print "# {} from {}".format(path)
        print ''.join(lines[start:])
        success += 1
        
    except Exception as err:
        print "# ⚠️", reaction, "Gave error", str(err)
        fail += 1
```